In [4]:
import docker
import progproxy as pp
client = docker.DockerClient(base_url='unix://var/run/docker.sock')


environment = {
    'PYTHONPATH':'/opt/libraries',
    'CONFIGPROXY_AUTH_TOKEN':"'31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"
}

# Set up the first cloudvolume container
volumes1 = {
    '/jukebox/LightSheetData/atlas/neuroglancer/atlas/princetonmouse':{
        'bind':'/mnt/data',
        'mode':'ro'
    },
    '/home/ahoag/Git/nglancer-frame/libraries':{
        'bind':'/opt/libraries'
    }
}

container1 = client.containers.run('cloudv_test',
                                  volumes=volumes1,
                                  environment=environment,
                                  network='nglancer',
                                  name='cloudv1',
                                  detach=True)
# Register the cloudvolume with the reverse proxy
proxy_h = pp.progproxy(target_hname='confproxy')
proxy_h.addroute(proxypath='cloudv1',proxytarget="http://cloudv1:1337")


# Set up the second cloudvolume container
volumes2 = {
    '/home/ahoag/ngdemo/demo_bucket/grin_lens/oostland_m27':{
        'bind':'/mnt/data',
        'mode':'ro'
    },
    '/home/ahoag/Git/nglancer-frame/libraries':{
        'bind':'/opt/libraries'
    }
}


container2 = client.containers.run('cloudv_test',
                                  volumes=volumes2,
                                  environment=environment,
                                  network='nglancer',
                                  name='cloudv2',
                                  detach=True)
# Register the cloudvolume with the reverse proxy
proxy_h.addroute(proxypath='cloudv2',proxytarget="http://cloudv2:1337")

2020-01-21 16:50:38,802 - progproxy - DEBUG - Programable Proxy Object Created
2020-01-21 16:50:38,803 - progproxy - DEBUG - Target set to http://confproxy:8001
2020-01-21 16:50:38,804 - progproxy - DEBUG - header set to {'Authorization': "token '31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"}
2020-01-21 16:50:38,814 - progproxy - DEBUG - adding path cloudv1 pointing to http://cloudv1:1337
2020-01-21 16:50:38,820 - progproxy - DEBUG - path added successfully
2020-01-21 16:50:40,352 - progproxy - DEBUG - adding path cloudv2 pointing to http://cloudv2:1337
2020-01-21 16:50:40,357 - progproxy - DEBUG - path added successfully


In [ ]:
import requests

In [7]:
header = {'Authorization': "token '31507a9ddf3e41cf86b58ffede2db68326657437704461ae2c1a4018d55e18f0'"}
response1 = requests.get('http://confproxy:8000/cloudv1/info',headers=header)
print(response1.text)
print()
response2 = requests.get('http://confproxy:8000/cloudv2/info',headers=header)
print(response2.text)

{
  "data_type": "uint16",
  "mesh": "mesh_mip_0_err_40",
  "num_channels": 1,
  "scales": [
    {
      "chunk_sizes": [
        [
          1024,
          1024,
          1
        ]
      ],
      "encoding": "raw",
      "key": "40000_40000_40000",
      "resolution": [
        40000,
        40000,
        40000
      ],
      "size": [
        352,
        640,
        540
      ],
      "voxel_offset": [
        0,
        0,
        1
      ]
    }
  ],
  "type": "segmentation"
}

{
  "data_type": "uint16",
  "num_channels": 1,
  "scales": [
    {
      "chunk_sizes": [
        [
          1024,
          1024,
          1
        ]
      ],
      "encoding": "raw",
      "key": "20000_20000_20000",
      "resolution": [
        20000,
        20000,
        20000
      ],
      "size": [
        352,
        640,
        540
      ],
      "voxel_offset": [
        0,
        0,
        0
      ]
    }
  ],
  "type": "image"
}


In [9]:
response=requests.get('http://confproxy:8001/api/routes',headers=header)
response.text

'{"/viewer0":{"target":"http://nglancer:8080/","last_activity":"2020-01-21T21:45:22.666Z"},"/testvol":{"target":"http://host.docker.internal:1341","last_activity":"2020-01-17T22:15:22.362Z"},"/vols/testvol":{"target":"http://localhost:1340","last_activity":"2020-01-17T21:07:14.835Z"},"/testvol2":{"target":"http://mytesthost:1341","last_activity":"2020-01-17T21:51:36.661Z"},"/testvol3":{"target":"http://mytestname:1341","last_activity":"2020-01-21T21:04:00.773Z"},"/cloudv1":{"target":"http://cloudv1:1337","last_activity":"2020-01-21T21:52:00.174Z"},"/cloudv2":{"target":"http://cloudv2:1337","last_activity":"2020-01-21T21:52:00.217Z"}}'